In [1]:
import os
import cv2
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import FileLink, display

INPUT_DIR = "/kaggle/input/tattoos"
image_paths = [os.path.join(INPUT_DIR, path) for path in os.listdir(INPUT_DIR)]
image_paths.sort()

In [2]:
def download_file(folder="/kaggle/working"):
    command = f"zip out.zip {folder} -r"
    subprocess.run(command, shell=True)


def clear_dir(folder="/kaggle/working"):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                clear_dir(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)


def visualize(*imgs):
    for i in range(len(imgs)):
        plt.subplot(2, 2, i + 1)
        plt.imshow(imgs[i], 'gray')

def remove_background_internal(image_paths):
    os.mkdir('internal')
    for path in image_paths:
        image = cv2.imread(path, 1)
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        image_mean = round(cv2.mean(image_grey)[0])
        thresh_algo = cv2.THRESH_BINARY if image_mean < 127 else cv2.THRESH_BINARY_INV
        _, image_thresh = cv2.threshold(image_grey, image_mean, 255, thresh_algo)
        
        final_image = cv2.merge([*cv2.split(image), image_thresh], 4)
        cv2.imwrite(f"internal/{path.split('/')[-1]}", final_image)

        
def remove_background_external(image_paths):
    os.mkdir('external')
    for path in image_paths[:5]:
        image = cv2.imread(path, 1)
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        image_mean = round(cv2.mean(image_grey)[0]) * 0.8
        print(image_mean)
        thresh_algo = cv2.THRESH_BINARY if image_mean < 127 else cv2.THRESH_BINARY_INV
        _, image_thresh = cv2.threshold(image_grey, image_mean, 255, thresh_algo)
        
        contours, _ = cv2.findContours(image=image_thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
        contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)[:5]
        
        mask = np.zeros(image.shape[:2], np.uint8)
        cv2.drawContours(mask, contours, -1, 255, -1)

        final_image = cv2.merge([*cv2.split(image), mask], 4)
        cv2.imwrite(f"external/{path.split('/')[-1]}", final_image)
        

# clear_dir()
# remove_background_internal(image_paths)
# remove_background_external(image_paths)
# download_file()